In [1]:
from utils.load_results import *
from utils.plot_helpers import *

import pandas as pd
#import seaborn as sns
from matplotlib import pyplot as plt
plt.style.use('default')
import torch
from utils.analysis_from_interaction import *
from language_analysis_local import TopographicSimilarityConceptLevel, encode_target_concepts_for_topsim
import os
if not os.path.exists('analysis'):
    os.makedirs('analysis')
#import plotly.express as px

In [328]:
datasets = ['(3,4)', '(3,8)', '(3,16)', '(4,4)', '(4,8)', '(5,4)']
n_values = [4, 8, 16, 4, 8, 4]
n_attributes = [3, 3, 3, 4, 4, 5]
n_epochs = 300
n_datasets = len(datasets)
paths = ['results/' + d + '_game_size_10_vsf_3' for d in datasets]

In [7]:
datasets = ['(3,4)']
n_values = [4,]
n_attributes = [3,]
n_epochs = 300
n_datasets = len(datasets)
paths = ['results/' + d + '_game_size_10_vsf_3' for d in datasets]

In [6]:
context_unaware = False # whether original or context_unaware simulations are evaluated

setting = 'standard'
granularity_list = ['coarse', 'mixed', 'fine']
non_default_gran_list = ['coarse', 'fine']

### Determine vocab size

In [ ]:
import os
import csv
from collections import Counter

# Define the directory and filename for the CSV file
output_folder = 'analysis'
csv_filename = os.path.join(output_folder, '00_vocab_sizes.csv')

# Check if the output folder exists, if not, create it
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Check if the CSV file exists, if not, create it with headers
fieldnames = ['dataset', 'context_condition', 'vocab_size', 'symbols']
file_exists = os.path.exists(csv_filename)
with open(csv_filename, 'a', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    if not file_exists:
        writer.writeheader()

# Open the file in append mode and write the vocab sizes for each dataset
with open(csv_filename, 'a', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    # Go through all datasets
    for i, d in enumerate(datasets):
        for g in granularity_list:
            # select first run
            if g != 'mixed':
                path_to_run = paths[i] + '/' + str(setting) + '/' + 'granularity_' + g + '/' + str(0) + '/'
                path_to_interaction_train = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
                interaction = torch.load(path_to_interaction_train)
                
                messages = interaction.message.argmax(dim=-1)
                messages = [msg.tolist() for msg in messages]
                all_symbols = [symbol for message in messages for symbol in message]
                symbol_counts = Counter(all_symbols)
                vocab_size = len(symbol_counts)

            elif g == 'mixed':
                path_to_run = paths[i] + '/' + str(setting) + '/' + str(0) + '/'
                path_to_interaction_train = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
                interaction = torch.load(path_to_interaction_train)
                
                messages = interaction.message.argmax(dim=-1)
                messages = [msg.tolist() for msg in messages]
                all_symbols = [symbol for message in messages for symbol in message]
                symbol_counts = Counter(all_symbols)
                vocab_size = len(symbol_counts)
            
            writer.writerow({'dataset': d, 'context_condition': g, 'vocab_size': vocab_size, 'symbols': symbol_counts})


In [32]:
import pandas as pd
df = pd.read_csv('analysis/00_vocab_sizes.csv')
df

,dataset,context_condition,vocab_size,symbols
0,"(3,4)",coarse,12,"Counter({0: 742, 14: 450, 9: 391, 4: 387, 6: 2..."
1,"(3,4)",mixed,15,"Counter({0: 1807, 6: 779, 2: 582, 8: 537, 11: ..."
2,"(3,4)",fine,14,"Counter({0: 742, 14: 332, 5: 315, 4: 214, 12: ..."
3,"(3,8)",coarse,17,"Counter({0: 4364, 16: 1585, 6: 1552, 17: 1485,..."
4,"(3,8)",mixed,28,"Counter({0: 11662, 25: 4555, 2: 2113, 12: 2050..."
5,"(3,8)",fine,27,"Counter({0: 4364, 23: 1292, 16: 947, 4: 872, 2..."
6,"(3,16)",coarse,32,"Counter({0: 29467, 18: 15057, 47: 10339, 41: 6..."
7,"(3,16)",mixed,52,"Counter({0: 83302, 47: 18222, 12: 13188, 37: 1..."
8,"(3,16)",fine,52,"Counter({0: 29467, 5: 8383, 22: 5202, 19: 4557..."
9,"(4,4)",coarse,15,"Counter({0: 3743, 4: 2910, 14: 2438, 15: 1895,..."


### See which messages are produced for which concepts

In [4]:
datasets = ['(3,4)',]
n_values = [4]
n_attributes = [3]
n_epochs = 300
n_datasets = len(datasets)
paths = ['results/' + d + '_game_size_10_vsf_3' for d in datasets]

In [ ]:
# go through all datasets
for i, d in enumerate(datasets):
    for g in non_default_gran_list:
        if g != 'mixed':
        # select first run
            path_to_run = paths[i] + '/' + str(setting) +'/granularity_' + g +'/' + str(0) + '/'
        else:
            path_to_run = paths[i] + '/' + str(setting) +'/' + str(0) + '/'
            
        path_to_interaction_train = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
        path_to_interaction_val = (path_to_run + 'interactions/validation/epoch_' + str(n_epochs) + '/interaction_gpu0')
        interaction = torch.load(path_to_interaction_train)
        print(path_to_interaction_train)

        messages = interaction.message.argmax(dim=-1)
        messages = [msg.tolist() for msg in messages]
        sender_input = interaction.sender_input
        print(sender_input.shape)
        n_targets = int(sender_input.shape[1]/2)
        # get target objects and fixed vectors to re-construct concepts
        target_objects = sender_input[:, :n_targets]
        target_objects = k_hot_to_attributes(target_objects, n_values[i])
        # concepts are defined by a list of target objects (here one sampled target object) and a fixed vector
        (objects, fixed) = retrieve_concepts_sampling(target_objects, all_targets=True)
        concepts = list(zip(objects, fixed))

        # get distractor objects to re-construct context conditions
        distractor_objects = sender_input[:, n_targets:]
        distractor_objects = k_hot_to_attributes(distractor_objects, n_values[i])
        context_conds = retrieve_context_condition(objects, fixed, distractor_objects)
        #print(context_conds)

        # get random qualitative samples
        #fixed_index = random.randint(0, n_attributes[i]-1) # define a fixed index for the concept
        n_fixed = random.randint(1, n_attributes[i]) # how many fixed attributes?
        #n_fixed = 3
        fixed_indices = random.sample(range(0, n_attributes[i]), k=n_fixed) # select which attributes are fixed
        #fixed_indices = [0, 2, 1]
        #fixed_value = random.randint(0, n_values[i]-1) # define a fixed value for this index
        fixed_values = random.choices(range(0, n_values[i]), k=n_fixed)
        #fixed_values = [0, 1, 2]
        print(n_fixed, fixed_indices, fixed_values)
        #index_threshold = 20000 # optional: define some index threshold to make sure that examples are not taken from the beginning of training
        # TODO: adapt this loop such that multiple indices can be fixed
        all_for_this_concept = []
        for idx, (t_objects, t_fixed) in enumerate(concepts):
            #if sum(t_fixed) == 1 and t_fixed[fixed_index] == 1:# and idx > index_threshold:
            if sum(t_fixed) == n_fixed and all(t_fixed[fixed_index] == 1 for fixed_index in fixed_indices):
                for t_object in t_objects:
                    if all(t_object[fixed_index] == fixed_values[j] for j, fixed_index in enumerate(fixed_indices)):
                        all_for_this_concept.append((idx, t_object, t_fixed, context_conds[idx], messages[idx]))
                        fixed = t_fixed
        #print(all_for_this_concept)                
        if len(all_for_this_concept) > 0:
            #sample = random.sample(all_for_this_concept, 20)
            sample = all_for_this_concept
            column_names = ['game_nr', 'object', 'fixed indices', 'context condition', 'message']
            df = pd.DataFrame(sample, columns=column_names)
            df.to_csv('analysis/quali_' + str(d) + '_' + str(setting) + '_'+g +'_' + str(sample[0][1]) + ',' + str(fixed) + 'all.csv', index=False)
            print('saved ' + 'analysis/quali_' + str(d) + '_' + str(setting) +'_' + g + '_' + str(sample[0][1]) + ',' + str(fixed) + 'all.csv')
        else:
            raise ValueError("sample for dataset " + str(d) + g + " could not be generated")

In [29]:
import pandas as pd
df = pd.read_csv('analysis/quali_(3,4)_standard_[0. 3. 2.],[1. 1. 1.].csv')
df

,game_nr,object,fixed indices,context condition,message
0,1384,[0. 3. 2.],[1. 1. 1.],2,"[1, 1, 1, 0]"
1,90,[0. 3. 2.],[1. 1. 1.],1,"[1, 1, 1, 0]"
2,1384,[0. 3. 2.],[1. 1. 1.],2,"[1, 1, 1, 0]"
3,1183,[0. 3. 2.],[1. 1. 1.],1,"[11, 1, 1, 0]"
4,1798,[0. 3. 2.],[1. 1. 1.],0,"[11, 1, 9, 0]"
5,1483,[0. 3. 2.],[1. 1. 1.],2,"[1, 1, 14, 0]"
6,389,[0. 3. 2.],[1. 1. 1.],0,"[1, 1, 14, 0]"
7,1535,[0. 3. 2.],[1. 1. 1.],0,"[11, 1, 1, 0]"
8,1535,[0. 3. 2.],[1. 1. 1.],0,"[11, 1, 1, 0]"
9,1699,[0. 3. 2.],[1. 1. 1.],1,"[1, 1, 9, 0]"


### Count messages used for each dataset

In [ ]:
import csv
import os
import torch

# Define the directory and filename for the CSV file
analysis_folder = 'analysis'
csv_filename = os.path.join(analysis_folder, 'message_counts.csv')

fieldnames = ['dataset', 'context_condition', 'unique_messages']

# Check if the CSV file exists, if not, create it with headers
if not os.path.exists(csv_filename):
    with open(csv_filename, 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

# Go through all datasets
for i, d in enumerate(datasets):
    for g in granularity_list:
        if g != 'mixed':
      # select first run
            path_to_run = paths[i] + '/' + str(setting) + '/granularity_' + g + '/' + str(0) + '/'
        else:
            path_to_run = paths[i] + '/' + str(setting) +'/' + str(0) + '/'

        path_to_interaction_train = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
        path_to_interaction_val = (path_to_run + 'interactions/validation/epoch_' + str(n_epochs) + '/interaction_gpu0')
        interaction = torch.load(path_to_interaction_train)
        print(path_to_interaction_train)

        messages = interaction.message.argmax(dim=-1)
        messages = [msg.tolist() for msg in messages]
        unique_messages = set(tuple(msg) for msg in messages)  # Convert lists to tuples before adding to the set

        mess_count = len(unique_messages)
        # Check if the combination of dataset and context condition already exists in the CSV file
        combination_exists = False
        with open(csv_filename, 'r', newline='') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                if row['dataset'] == d and row['context_condition'] == g:
                    combination_exists = True
                    break

        # If the combination exists, update the corresponding row
        if combination_exists:
            rows = []
            with open(csv_filename, 'r', newline='') as csvfile:
                reader = csv.DictReader(csvfile)
                for row in reader:
                    if row['dataset'] == d and row['context_condition'] == g:
                        row['unique_messages'] = mess_count
                    rows.append(row)
            
            with open(csv_filename, 'w', newline='') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(rows)
        # If the combination does not exist, append a new row
        else:
            with open(csv_filename, 'a', newline='') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                writer.writerow({'dataset': d, 'context_condition': g, 'unique_messages': mess_count})


In [35]:
import pandas as pd
df = pd.read_csv('analysis/message_counts.csv')
df

,dataset,context_condition,unique_messages
0,"(3,4)",coarse,33
1,"(3,4)",mixed,175
2,"(3,4)",fine,133
3,"(3,8)",coarse,41
4,"(3,8)",mixed,803
5,"(3,8)",fine,438
6,"(3,16)",coarse,133
7,"(3,16)",mixed,4032
8,"(3,16)",fine,1308
9,"(4,4)",coarse,204


## Concepts and messages

### Explorative analysis

In [36]:
import sklearn

In [145]:
datasets = ['(3,4)']
n_values = [4]
n_attributes = [4]
n_epochs = 300
n_datasets = len(datasets)
paths = ['results/' + d + '_game_size_10_vsf_3' for d in datasets]
setting = 'standard'

granularity = 'coarse'

In [ ]:
# go through all datasets
for i, d in enumerate(datasets):
        # select first run
        path_to_run = paths[i] + '/' + str(setting) +'/granularity_' + granularity +'/' + str(0) + '/'
            
        path_to_interaction_train = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
        path_to_interaction_val = (path_to_run + 'interactions/validation/epoch_' + str(n_epochs) + '/interaction_gpu0')
        interaction = torch.load(path_to_interaction_train)
        print(path_to_interaction_train)

        messages = interaction.message.argmax(dim=-1)
        messages = [msg.tolist() for msg in messages]
        sender_input = interaction.sender_input
        print(sender_input.shape)
        n_targets = int(sender_input.shape[1]/2)
        # get target objects and fixed vectors to re-construct concepts
        target_objects = sender_input[:, :n_targets]
        target_objects = k_hot_to_attributes(target_objects, n_values[i])
        # concepts are defined by a list of target objects (here one sampled target object) and a fixed vector
        (objects, fixed) = retrieve_concepts_sampling(target_objects, all_targets=True)
        concepts = list(zip(objects, fixed))

        # get distractor objects to re-construct context conditions
        distractor_objects = sender_input[:, n_targets:]
        distractor_objects = k_hot_to_attributes(distractor_objects, n_values[i])
        context_conds = retrieve_context_condition(objects, fixed, distractor_objects)
        #print(context_conds)

        # get random qualitative samples
        #fixed_index = random.randint(0, n_attributes[i]-1) # define a fixed index for the concept
        #n_fixed = random.randint(1, n_attributes[i]) # how many fixed attributes?
        n_fixed =1
        #fixed_indices = random.sample(range(0, n_attributes[i]), k=n_fixed) # select which attributes are fixed
        fixed_indices = [0]
        #fixed_value = random.randint(0, n_values[i]-1) # define a fixed value for this index
        #fixed_values = random.choices(range(0, n_values[i]), k=n_fixed)
        fixed_values = [2]
        print(n_fixed, fixed_indices, fixed_values)
        #index_threshold = 20000 # optional: define some index threshold to make sure that examples are not taken from the beginning of training
        # TODO: adapt this loop such that multiple indices can be fixed
        all_for_this_concept = []
        for idx, (t_objects, t_fixed) in enumerate(concepts):
            #if sum(t_fixed) == 1 and t_fixed[fixed_index] == 1:# and idx > index_threshold:
            if sum(t_fixed) == n_fixed and all(t_fixed[fixed_index] == 1 for fixed_index in fixed_indices):
                for t_object in t_objects:
                    if all(t_object[fixed_index] == fixed_values[j] for j, fixed_index in enumerate(fixed_indices)):
                        all_for_this_concept.append((idx, t_object, t_fixed, context_conds[idx], messages[idx]))
                        fixed = t_fixed
        #print(all_for_this_concept)                
        if len(all_for_this_concept) > 0:
            #sample = random.sample(all_for_this_concept, 20)
            sample = all_for_this_concept
            column_names = ['game_nr', 'object', 'fixed indices', 'context condition', 'message']
            df = pd.DataFrame(sample, columns=column_names)
            df.to_csv('analysis/quali_' + str(d) + '_' + str(setting) + '_'+granularity +'_' + str(sample[0][1]) + ',' + str(fixed) + 'all.csv', index=False)
            print('saved ' + 'analysis/quali_' + str(d) + '_' + str(setting) +'_' + granularity+ '_' + str(sample[0][1]) + ',' + str(fixed) + 'all.csv')
        else:
            raise ValueError("sample for dataset " + str(d) + granularity + " could not be generated")

### Concepts for given messages in coarse context condition

In [155]:
def create_alternative_representation(concept):
        obj, fixed = concept
        new_obj = [
            value if is_fixed else '?' 
            for value, is_fixed in zip(obj, fixed)
        ]
        rep = (((new_obj), (fixed)))

        return rep



In [246]:
import ast
def concepts_for_message(df, messages):
    concepts = []
    for message in messages:
        ex = df[df['message'] == str(message)]

        # Extract the concepts from the matching row
        concepts_str = ex.iloc[0, 1]
        # Convert the string representation of the tuple of lists to a Python object (tuple of lists)
        concepts_tuple_of_lists = ast.literal_eval(concepts_str)
        

        # Print each tuple of lists on a new line
        for concept_tuple in set(concepts_tuple_of_lists):
            concepts.append(concept_tuple)
    return(concepts)
        

In [274]:
import torch
import pandas as pd
import random

# Assuming these variables are already defined: datasets, paths, setting, non_default_gran_list, n_epochs, n_values, n_attributes


# Go through all datasets
for i, d in enumerate(datasets):
    path_to_run = paths[i] + '/' + str(setting) + '/granularity_coarse' + '/' + str(0) + '/'
    
    path_to_interaction_train = path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0'
    path_to_interaction_val = path_to_run + 'interactions/validation/epoch_' + str(n_epochs) + '/interaction_gpu0'
    interaction = torch.load(path_to_interaction_train)
    #print("Interaction train path:", path_to_interaction_train)

    messages = interaction.message.argmax(dim=-1)
    messages = [msg.tolist() for msg in messages]
    sender_input = interaction.sender_input
    #print("Sender input shape:", sender_input.shape)
    n_targets = int(sender_input.shape[1] / 2)
    
    # Get target objects and fixed vectors to re-construct concepts
    target_objects = sender_input[:, :n_targets]
    target_objects = k_hot_to_attributes(target_objects, n_values[i])
    # Concepts are defined by a list of target objects (here one sampled target object) and a fixed vector
    (objects, fixed) = retrieve_concepts_sampling(target_objects, all_targets=True)
    concepts = list(zip(objects, fixed))
    #print(concepts[0])

   # Collect data for the DataFrame

    mess = []
    concept_reps= []

    for idx, (t_objects, t_fixed) in enumerate(concepts):
        #concept_sample = (tuple(t_objects[0]), tuple(t_fixed))
        concept_sample = (
                list(int(x) for x in t_objects[0]), 
                list(int(x) for x in t_fixed)
            )
        mess.append(tuple(messages[idx]))
        concept_reps.append(create_alternative_representation(concept_sample))

        
        

    # Create the DataFrame
    df = pd.DataFrame({'concepts': concept_reps, 'message': mess})


    # Group by messages and aggregate concepts
    message_df = df.groupby('message', as_index=False).agg({'concepts': lambda x: list(x)})
    synonym_df = df.groupby('concepts', as_index=False).agg({'message' :lambda x:list(x)})
    synonym_df['message'] = synonym_df['message'].apply(lambda x: str(list(set(x))))

    

    # Display the DataFrame
    print(message_df.shape)
    print(synonym_df.shape)
    message_df.to_csv('analysis/messages/'+str(d) + '_' + str(setting) + '_granularity_coarse.csv', index=False)
    
    synonym_df.to_csv('analysis/messages/'+'synonyms_'+str(d) + '_' + str(setting) + '_granularity_coarse.csv', index=False)

    print('\n')
    ## create clean dataframe, where each concepts appears only once per row and always with same sample for easier comparison


(33, 2)
(99, 2)


(41, 2)
(582, 2)


(133, 2)
(3929, 2)


(204, 2)
(499, 2)


(788, 2)
(5248, 2)


(1443, 2)
(2499, 2)




### D(3,4)

In [239]:
df = pd.read_csv('analysis/messages/(3,4)_standard_granularity_coarse.csv')
concepts = []
mess = []
lens = []
for row in df.itertuples():
    concepts.append(concepts_for_message(df, [row[1]]))
    lens.append(len(concepts_for_message(df, [row[1]])))
    mess.append(row[1])
    print(row[1], 'n_concepts', len(concepts_for_message(df, [row[1]])),)

res = pd.DataFrame({'message':mess, 'n_concepts': lens })
res.to_csv('analysis/messages/lens.csv', index=False)

(1, 1, 1, 0) n_concepts 7
(1, 1, 8, 0) n_concepts 1
(1, 1, 12, 0) n_concepts 1
(1, 2, 2, 0) n_concepts 1
(1, 7, 1, 0) n_concepts 1
(1, 12, 1, 0) n_concepts 1
(1, 12, 12, 0) n_concepts 1
(2, 1, 2, 0) n_concepts 1
(2, 2, 2, 0) n_concepts 5
(3, 3, 3, 0) n_concepts 8
(3, 6, 6, 0) n_concepts 1
(3, 9, 9, 0) n_concepts 1
(4, 4, 4, 0) n_concepts 17
(6, 6, 6, 0) n_concepts 14
(6, 6, 15, 0) n_concepts 2
(6, 15, 6, 0) n_concepts 3
(6, 15, 15, 0) n_concepts 1
(7, 1, 1, 0) n_concepts 1
(7, 7, 7, 0) n_concepts 2
(8, 1, 8, 0) n_concepts 1
(8, 8, 1, 0) n_concepts 1
(8, 8, 8, 0) n_concepts 2
(9, 9, 3, 0) n_concepts 1
(9, 9, 9, 0) n_concepts 20
(9, 9, 15, 0) n_concepts 1
(9, 15, 9, 0) n_concepts 1
(12, 1, 12, 0) n_concepts 1
(12, 12, 12, 0) n_concepts 2
(14, 14, 14, 0) n_concepts 20
(15, 6, 6, 0) n_concepts 3
(15, 9, 9, 0) n_concepts 2
(15, 9, 15, 0) n_concepts 1
(15, 15, 15, 0) n_concepts 6


In [198]:
# Load DataFrame from CSV file
df = pd.read_csv('analysis/messages/(3,4)_standard_granularity_coarse.csv')
for c in concepts_for_message(df, [(4,4,4,0),]):
    print(c)

### fixed 1 is 1st postion whaaaaat

((1, 3, '?'), (1, 1, 0))
((1, 1, 2), (1, 1, 1))
((1, 0, 2), (1, 1, 1))
((1, 0, '?'), (1, 1, 0))
((1, 1, 1), (1, 1, 1))
((1, 3, 0), (1, 1, 1))
((1, 1, 3), (1, 1, 1))
((1, 0, 1), (1, 1, 1))
((1, 3, 1), (1, 1, 1))
((1, 1, '?'), (1, 1, 0))
((1, '?', 1), (1, 0, 1))
((1, 3, 3), (1, 1, 1))
((1, 3, 2), (1, 1, 1))
((1, '?', '?'), (1, 0, 0))
((1, '?', 0), (1, 0, 1))
((1, 1, 0), (1, 1, 1))
((1, 0, 0), (1, 1, 1))


In [199]:
import ast
import pandas as pd

# Load DataFrame from CSV file
df = pd.read_csv('analysis/messages/(3,4)_standard_granularity_coarse.csv')
for c in concepts_for_message(df, [(14,14,14,0),]):
    print(c)


## it looks loke this message referes to fixed 2 in second position whaaaaat

((0, 2, 3), (1, 1, 1))
((0, 2, '?'), (1, 1, 0))
((2, 2, 0), (1, 1, 1))
(('?', 2, 1), (0, 1, 1))
((1, 2, 2), (1, 1, 1))
((2, 2, 1), (1, 1, 1))
((0, 2, 1), (1, 1, 1))
(('?', 2, 3), (0, 1, 1))
((1, 2, '?'), (1, 1, 0))
(('?', 2, '?'), (0, 1, 0))
((2, 2, '?'), (1, 1, 0))
((1, 2, 0), (1, 1, 1))
((3, 2, 1), (1, 1, 1))
((0, 2, 0), (1, 1, 1))
((3, 2, 0), (1, 1, 1))
((2, 2, 2), (1, 1, 1))
((3, 2, 2), (1, 1, 1))
((3, 2, '?'), (1, 1, 0))
((3, 2, 3), (1, 1, 1))
((1, 2, 1), (1, 1, 1))


In [200]:

# Load DataFrame from CSV file
df = pd.read_csv('analysis/messages/(3,4)_standard_granularity_coarse.csv')
for c in concepts_for_message(df, [(9,9,9,0),]):
    print(c)
## it looks loke this message referes to fixed 0 in first position whaaaaat

((0, 2, 3), (1, 1, 1))
((0, 0, '?'), (1, 1, 0))
((0, 1, 3), (1, 1, 1))
((0, 1, 0), (1, 1, 1))
((0, 0, 0), (1, 1, 1))
((0, 3, 1), (1, 1, 1))
((0, 3, 0), (1, 1, 1))
((0, '?', 0), (1, 0, 1))
((0, 1, 1), (1, 1, 1))
((0, 3, 3), (1, 1, 1))
((0, 0, 2), (1, 1, 1))
((0, 1, 2), (1, 1, 1))
((0, 1, '?'), (1, 1, 0))
((0, 3, 2), (1, 1, 1))
((0, '?', 2), (1, 0, 1))
((0, '?', 1), (1, 0, 1))
((0, 3, '?'), (1, 1, 0))
((0, '?', 3), (1, 0, 1))
((0, 0, 1), (1, 1, 1))
((0, '?', '?'), (1, 0, 0))


In [286]:
import ast
import pandas as pd

# Load DataFrame from CSV file
df = pd.read_csv('analysis/messages/(3,4)_standard_granularity_coarse.csv')

for c in concepts_for_message(df, [(15,15,15,0),]):
    print(c)

## it looks loke this message referes to fixed 0 in second position whaaaaat

((3, 0, '?'), (1, 1, 0))
((2, 0, '?'), (1, 1, 0))
((2, 0, 1), (1, 1, 1))
(('?', 0, 0), (0, 1, 1))
(('?', 0, '?'), (0, 1, 0))
((3, 0, 3), (1, 1, 1))


In [201]:
import ast
import pandas as pd

# Load DataFrame from CSV file
df = pd.read_csv('analysis/messages/(3,4)_standard_granularity_coarse.csv')
for c in concepts_for_message(df, [(2,2,2,0),]):
    print(c)

## it looks loke this message referes to fixed 0 in third position whaaaaat

((3, 3, 0), (1, 1, 1))
((3, '?', 0), (1, 0, 1))
(('?', 3, 0), (0, 1, 1))
((3, 1, 0), (1, 1, 1))
(('?', '?', 0), (0, 0, 1))


In [202]:
import ast
import pandas as pd

# Load DataFrame from CSV file
df = pd.read_csv('analysis/messages/(3,4)_standard_granularity_coarse.csv')
for c in concepts_for_message(df, [(7,7,7,0),]):
    print(c)
## it looks loke this message referes to fixed 3 in first position whaaaaat

((3, '?', 2), (1, 0, 1))
((3, '?', '?'), (1, 0, 0))


In [248]:
import ast
import pandas as pd

# Load DataFrame from CSV file
df = pd.read_csv('analysis/messages/(3,4)_standard_granularity_coarse.csv')
for c in concepts_for_message(df, [(6,6,6,0),]):
    print(c)


## it looks like ALL THESE MESSAGES referes to fixed 2 in first position AND fixed 0 in second position!!!!!!!!!!!!!!!!!!!!

((2, '?', 2), (1, 0, 1))
((2, 0, 0), (1, 1, 1))
((2, 3, 0), (1, 1, 1))
((2, '?', 3), (1, 0, 1))
((2, 1, 2), (1, 1, 1))
((2, 0, '?'), (1, 1, 0))
((2, 3, '?'), (1, 1, 0))
((2, 0, 2), (1, 1, 1))
((2, 3, 1), (1, 1, 1))
((2, '?', 0), (1, 0, 1))
((2, 1, '?'), (1, 1, 0))
((2, 0, 3), (1, 1, 1))
((2, 3, 3), (1, 1, 1))
((2, 1, 0), (1, 1, 1))


In [249]:
df = pd.read_csv('analysis/messages/(3,4)_standard_granularity_coarse.csv')
for c in concepts_for_message(df, [(6, 6, 15, 0), (6, 15, 6, 0),(6, 15, 15, 0)]):
    print(c)

((2, 0, '?'), (1, 1, 0))
((2, 0, 2), (1, 1, 1))
((2, 0, '?'), (1, 1, 0))
((2, 0, 3), (1, 1, 1))
((2, 0, 2), (1, 1, 1))
((2, 0, 2), (1, 1, 1))


### D(3,8)

In [240]:
df = pd.read_csv('analysis/messages/(3,8)_standard_granularity_coarse.csv')
concepts = []
mess = []
lens = []
for row in df.itertuples():
    concepts.append(concepts_for_message(df, [row[1]]))
    lens.append(len(concepts_for_message(df, [row[1]])))
    mess.append(row[1])
    print(row[1], 'n_concepts', len(concepts_for_message(df, [row[1]])))

(5, 5, 5, 0) n_concepts 7
(5, 12, 12, 0) n_concepts 5
(5, 12, 21, 0) n_concepts 2
(5, 17, 5, 0) n_concepts 1
(5, 21, 21, 0) n_concepts 13
(6, 6, 6, 0) n_concepts 71
(9, 9, 9, 0) n_concepts 64
(12, 9, 9, 0) n_concepts 1
(12, 12, 12, 0) n_concepts 15
(12, 13, 13, 0) n_concepts 4
(13, 6, 13, 0) n_concepts 1
(13, 13, 13, 0) n_concepts 64
(13, 21, 13, 0) n_concepts 1
(13, 21, 24, 0) n_concepts 1
(15, 15, 15, 0) n_concepts 1
(16, 16, 16, 0) n_concepts 64
(17, 17, 17, 0) n_concepts 64
(18, 18, 9, 0) n_concepts 1
(18, 18, 18, 0) n_concepts 60
(21, 7, 7, 0) n_concepts 2
(21, 8, 8, 0) n_concepts 3
(21, 9, 9, 0) n_concepts 1
(21, 13, 7, 0) n_concepts 1
(21, 13, 13, 0) n_concepts 1
(21, 15, 15, 0) n_concepts 1
(21, 15, 21, 0) n_concepts 1
(21, 21, 19, 0) n_concepts 1
(21, 21, 21, 0) n_concepts 2
(21, 24, 7, 0) n_concepts 7
(21, 24, 24, 0) n_concepts 2
(23, 12, 12, 0) n_concepts 1
(23, 23, 23, 0) n_concepts 62
(24, 8, 8, 0) n_concepts 1
(24, 13, 13, 0) n_concepts 1
(24, 16, 16, 0) n_concepts 8
(24,

In [252]:
df = pd.read_csv('analysis/messages/(3,8)_standard_granularity_coarse.csv')
for c in concepts_for_message(df, [(23,23,23,0),]):
    print(c)
#fixed 5 in first position

((5, 0, 5), (1, 1, 1))
((5, 7, 0), (1, 1, 1))
((5, '?', 0), (1, 0, 1))
((5, 1, 0), (1, 1, 1))
((5, 3, '?'), (1, 1, 0))
((5, 2, 6), (1, 1, 1))
((5, 7, 6), (1, 1, 1))
((5, 0, 0), (1, 1, 1))
((5, 3, 4), (1, 1, 1))
((5, 2, 5), (1, 1, 1))
((5, '?', 7), (1, 0, 1))
((5, 4, 7), (1, 1, 1))
((5, 0, 6), (1, 1, 1))
((5, 3, 7), (1, 1, 1))
((5, 5, 1), (1, 1, 1))
((5, 3, 2), (1, 1, 1))
((5, 0, 1), (1, 1, 1))
((5, 6, 2), (1, 1, 1))
((5, 1, 7), (1, 1, 1))
((5, 2, 1), (1, 1, 1))
((5, 7, 1), (1, 1, 1))
((5, 0, 2), (1, 1, 1))
((5, 4, 5), (1, 1, 1))
((5, 7, 4), (1, 1, 1))
((5, 6, 5), (1, 1, 1))
((5, 0, '?'), (1, 1, 0))
((5, 5, '?'), (1, 1, 0))
((5, 1, 3), (1, 1, 1))
((5, '?', 5), (1, 0, 1))
((5, 6, 7), (1, 1, 1))
((5, 2, '?'), (1, 1, 0))
((5, 0, 7), (1, 1, 1))
((5, 5, 7), (1, 1, 1))
((5, 3, 3), (1, 1, 1))
((5, 1, 5), (1, 1, 1))
((5, 0, 4), (1, 1, 1))
((5, 6, 1), (1, 1, 1))
((5, 1, 4), (1, 1, 1))
((5, 7, '?'), (1, 1, 0))
((5, 4, 1), (1, 1, 1))
((5, 7, 7), (1, 1, 1))
((5, 3, 5), (1, 1, 1))
((5, 6, 3), (1, 1,

In [253]:
df = pd.read_csv('analysis/messages/(3,8)_standard_granularity_coarse.csv')
for c in concepts_for_message(df, [(12,12,12,0),]):
    print(c)
# fixed 2 in second position

(('?', 2, 3), (0, 1, 1))
(('?', 2, 4), (0, 1, 1))
((5, 2, 3), (1, 1, 1))
((3, 2, '?'), (1, 1, 0))
(('?', 2, '?'), (0, 1, 0))
(('?', 2, 6), (0, 1, 1))
(('?', 2, 2), (0, 1, 1))
(('?', 2, 7), (0, 1, 1))
(('?', '?', 3), (0, 0, 1))
((3, 2, 3), (1, 1, 1))
(('?', 2, 0), (0, 1, 1))
(('?', 2, 1), (0, 1, 1))
((0, 2, 7), (1, 1, 1))
((0, 2, 0), (1, 1, 1))
((0, 2, 1), (1, 1, 1))


In [255]:
df = pd.read_csv('analysis/messages/(3,8)_standard_granularity_coarse.csv')
for c in concepts_for_message(df, [(23,12,12,0)]):
    print(c)
#works!

((5, 2, 3), (1, 1, 1))


In [256]:
df = pd.read_csv('analysis/messages/(3,8)_standard_granularity_coarse.csv')
for c in concepts_for_message(df, [(5,21,21,0)]):
    print(c)
 #1 in second pos

(('?', 1, 7), (0, 1, 1))
((7, 1, 4), (1, 1, 1))
(('?', 1, '?'), (0, 1, 0))
((7, 1, 3), (1, 1, 1))
((7, 1, '?'), (1, 1, 0))
((7, 1, 6), (1, 1, 1))
((7, 1, 2), (1, 1, 1))
((7, 1, 5), (1, 1, 1))
((7, 1, 0), (1, 1, 1))
(('?', 1, 6), (0, 1, 1))
(('?', 1, 5), (0, 1, 1))
(('?', 1, 1), (0, 1, 1))
((7, 1, 7), (1, 1, 1))


### D(3,16)

In [261]:
df = pd.read_csv('analysis/messages/(3,16)_standard_granularity_coarse.csv')
concepts = []
mess = []
lens = []
for row in df.itertuples():
    concepts.append(concepts_for_message(df, [row[1]]))
    lens.append(len(concepts_for_message(df, [row[1]])))
    mess.append(row[1])
    print(row[1], 'n_concepts', len(concepts_for_message(df, [row[1]])),)

res = pd.DataFrame({'message':mess, 'n_concepts': lens })
res.to_csv('analysis/messages/lens.csv', index=False)

(1, 1, 1, 0) n_concepts 1
(1, 1, 8, 0) n_concepts 65
(1, 1, 14, 0) n_concepts 19
(1, 1, 18, 0) n_concepts 213
(1, 18, 18, 0) n_concepts 1
(3, 3, 3, 0) n_concepts 235
(4, 27, 18, 0) n_concepts 1
(4, 27, 27, 0) n_concepts 21
(5, 5, 5, 0) n_concepts 226
(5, 5, 41, 0) n_concepts 1
(5, 23, 35, 0) n_concepts 1
(5, 42, 5, 0) n_concepts 3
(5, 42, 42, 0) n_concepts 1
(9, 9, 3, 0) n_concepts 3
(9, 9, 6, 0) n_concepts 2
(9, 9, 9, 0) n_concepts 240
(9, 9, 32, 0) n_concepts 5
(9, 9, 42, 0) n_concepts 2
(9, 32, 3, 0) n_concepts 3
(12, 12, 45, 0) n_concepts 81
(12, 45, 45, 0) n_concepts 221
(16, 27, 14, 0) n_concepts 6
(16, 27, 18, 0) n_concepts 13
(16, 27, 41, 0) n_concepts 1
(16, 41, 41, 0) n_concepts 3
(17, 47, 47, 0) n_concepts 41
(18, 9, 9, 0) n_concepts 1
(18, 17, 47, 0) n_concepts 1
(18, 18, 18, 0) n_concepts 228
(18, 47, 47, 0) n_concepts 235
(23, 23, 35, 0) n_concepts 1
(23, 35, 35, 0) n_concepts 245
(23, 35, 42, 0) n_concepts 1
(23, 42, 42, 0) n_concepts 1
(25, 1, 8, 0) n_concepts 1
(25, 1,

In [262]:
df = pd.read_csv('analysis/messages/(3,16)_standard_granularity_coarse.csv')
for c in concepts_for_message(df, [(9,9,9,0)]):
    print(c)
#10 first position

((10, 2, '?'), (1, 1, 0))
((10, 12, 5), (1, 1, 1))
((10, 2, 4), (1, 1, 1))
((10, 11, 13), (1, 1, 1))
((10, 1, 9), (1, 1, 1))
((10, 6, 2), (1, 1, 1))
((10, 9, 3), (1, 1, 1))
((10, 6, 1), (1, 1, 1))
((10, 8, 7), (1, 1, 1))
((10, 12, 6), (1, 1, 1))
((10, 14, 15), (1, 1, 1))
((10, 0, 12), (1, 1, 1))
((10, 14, 9), (1, 1, 1))
((10, 12, 1), (1, 1, 1))
((10, 6, 3), (1, 1, 1))
((10, 7, 15), (1, 1, 1))
((10, 3, 2), (1, 1, 1))
((10, 4, 10), (1, 1, 1))
((10, 2, 3), (1, 1, 1))
((10, 8, 8), (1, 1, 1))
((10, 9, 5), (1, 1, 1))
((10, 0, 14), (1, 1, 1))
((10, 2, 10), (1, 1, 1))
((10, 14, '?'), (1, 1, 0))
((10, 6, 5), (1, 1, 1))
((10, 13, 0), (1, 1, 1))
((10, 1, 8), (1, 1, 1))
((10, 5, 9), (1, 1, 1))
((10, 8, 3), (1, 1, 1))
((10, 2, 5), (1, 1, 1))
((10, 11, 15), (1, 1, 1))
((10, 14, 11), (1, 1, 1))
((10, 11, 0), (1, 1, 1))
((10, 1, 10), (1, 1, 1))
((10, 15, 5), (1, 1, 1))
((10, '?', 4), (1, 0, 1))
((10, 3, 15), (1, 1, 1))
((10, 4, 12), (1, 1, 1))
((10, 15, 7), (1, 1, 1))
((10, 10, 1), (1, 1, 1))
((10, 2,

### D(4,4)

In [263]:
df = pd.read_csv('analysis/messages/(4,4)_standard_granularity_coarse.csv')
concepts = []
mess = []
lens = []
for row in df.itertuples():
    concepts.append(concepts_for_message(df, [row[1]]))
    lens.append(len(concepts_for_message(df, [row[1]])))
    mess.append(row[1])
    print(row[1], 'n_concepts', len(concepts_for_message(df, [row[1]])),)

res = pd.DataFrame({'message':mess, 'n_concepts': lens })
res.to_csv('analysis/messages/lens.csv', index=False)

(2, 2, 2, 2, 0) n_concepts 6
(3, 3, 3, 3, 0) n_concepts 8
(3, 3, 6, 3, 0) n_concepts 2
(3, 3, 6, 6, 0) n_concepts 1
(3, 3, 7, 7, 0) n_concepts 1
(3, 3, 14, 14, 0) n_concepts 1
(3, 6, 3, 3, 0) n_concepts 1
(3, 14, 14, 14, 0) n_concepts 7
(4, 4, 4, 4, 0) n_concepts 98
(4, 4, 4, 7, 0) n_concepts 6
(4, 4, 4, 9, 0) n_concepts 2
(4, 4, 7, 4, 0) n_concepts 3
(4, 4, 9, 4, 0) n_concepts 2
(4, 4, 9, 9, 0) n_concepts 1
(4, 7, 4, 4, 0) n_concepts 5
(4, 7, 4, 7, 0) n_concepts 1
(4, 7, 7, 7, 0) n_concepts 2
(4, 9, 4, 4, 0) n_concepts 5
(4, 9, 9, 4, 0) n_concepts 2
(4, 10, 4, 4, 0) n_concepts 1
(5, 7, 7, 7, 0) n_concepts 2
(5, 7, 8, 8, 0) n_concepts 1
(5, 8, 8, 7, 0) n_concepts 1
(6, 2, 2, 2, 0) n_concepts 1
(6, 2, 6, 6, 0) n_concepts 1
(6, 3, 3, 6, 0) n_concepts 1
(6, 6, 2, 2, 0) n_concepts 1
(6, 6, 6, 2, 0) n_concepts 2
(6, 6, 6, 3, 0) n_concepts 1
(6, 6, 6, 6, 0) n_concepts 19
(6, 6, 6, 8, 0) n_concepts 7
(6, 6, 6, 11, 0) n_concepts 2
(6, 6, 6, 12, 0) n_concepts 1
(6, 6, 6, 14, 0) n_concepts 1
(6,

In [264]:
for c in concepts_for_message(df, [(4,4,4,4,0)]):
    print(c)
## 1 at the second place

(('?', 1, 0, '?'), (0, 1, 1, 0))
((1, 1, 3, 0), (1, 1, 1, 1))
((1, 1, '?', '?'), (1, 1, 0, 0))
((1, 1, 2, '?'), (1, 1, 1, 0))
((1, 1, 0, 1), (1, 1, 1, 1))
((3, 1, 2, 1), (1, 1, 1, 1))
((2, 1, 2, 1), (1, 1, 1, 1))
((3, 1, 2, 3), (1, 1, 1, 1))
((2, 1, '?', '?'), (1, 1, 0, 0))
((1, 1, 2, 2), (1, 1, 1, 1))
((1, 1, '?', 2), (1, 1, 0, 1))
(('?', 1, 3, 0), (0, 1, 1, 1))
((0, 1, 3, 2), (1, 1, 1, 1))
((3, 1, '?', 1), (1, 1, 0, 1))
(('?', 1, 3, 2), (0, 1, 1, 1))
(('?', 1, 2, 0), (0, 1, 1, 1))
((1, 1, 3, '?'), (1, 1, 1, 0))
((2, 1, 2, 0), (1, 1, 1, 1))
((0, 1, 1, 1), (1, 1, 1, 1))
((3, 1, 2, 2), (1, 1, 1, 1))
((2, 1, 2, 2), (1, 1, 1, 1))
((3, 1, '?', '?'), (1, 1, 0, 0))
((2, 1, 1, 0), (1, 1, 1, 1))
((1, 1, 3, 2), (1, 1, 1, 1))
((1, 1, 1, 2), (1, 1, 1, 1))
((2, 1, '?', 0), (1, 1, 0, 1))
((3, 1, 3, 1), (1, 1, 1, 1))
((0, 1, 1, 3), (1, 1, 1, 1))
((3, 1, '?', 2), (1, 1, 0, 1))
((3, 1, 3, 3), (1, 1, 1, 1))
((3, 1, 3, 2), (1, 1, 1, 1))
((1, 1, 0, '?'), (1, 1, 1, 0))
((3, 1, 2, '?'), (1, 1, 1, 0))
((3, 

### Lexicon coarse context condition

In [341]:
import os

# Specify the directory path
directory_path = "analysis/lexica"

# Check if the directory exists
if not os.path.exists(directory_path):
    # If it doesn't exist, create the directory
    os.makedirs(directory_path)
    print(f"Directory '{directory_path}' created successfully.")
else:
    print(f"Directory '{directory_path}' already exists.")

Directory 'analysis/lexica' created successfully.


In [386]:
for i, d in enumerate(datasets):
    df = pd.read_csv('analysis/messages/' + d + '_standard_granularity_coarse.csv')
    messages = []
    fixed_attribute_values = []
    symbols = []
    for row in df.itertuples():
        objs, fixed = [], []
        message = row[1]
        concepts = concepts_for_message(df, [message,])
        message = ast.literal_eval(message)
        symbol = (set(message[:-1]))
        print(message)
        messages.append(message)
        symbols.append(symbol)
        print(symbol)
        for obj, f in concepts:
            objs.append(obj)
            fixed.append(f)
        #print(objs)
        #print(fixed)
        idx_fixed_attrs = []
        for n in range(n_attributes[i]):
            nth_elements = [sublist[n] for sublist in fixed]
            if all(e == 1 for e in nth_elements):
                idx_fixed_attrs.append(n)
        #print(idx_fixed_attrs)
        fixed_values = ['_'] * n_attributes[i]  # List of n zeros
        for p in idx_fixed_attrs:  # Directly use p as index
            fixed_values[p] = objs[0][p]
            #print(objs[0][p])
        fixed_attribute_values.append(tuple(fixed_values))
    lexicon = pd.DataFrame({'message': messages, 'fixed attribute values': fixed_attribute_values})
    lexicon = lexicon.groupby('fixed attribute values', as_index=False).agg({'message': lambda x: list(x)})
    lexicon.to_csv('analysis/lexica/'+d+'_granularity_coarse.csv', index=False)
    symbol_lexicon = pd.DataFrame({'symbols': symbols, 'fixed attribute values': fixed_attribute_values})
    symbol_lexicon = symbol_lexicon.groupby('fixed attribute values', as_index=False).agg({'symbols': lambda x: list(x)})
    symbol_lexicon.to_csv('analysis/lexica/'+d+'symbols_granularity_coarse.csv', index=False)
      
        

(1, 1, 1, 0)
{1}
(1, 1, 8, 0)
{8, 1}
(1, 1, 12, 0)
{1, 12}
(1, 2, 2, 0)
{1, 2}
(1, 7, 1, 0)
{1, 7}
(1, 12, 1, 0)
{1, 12}
(1, 12, 12, 0)
{1, 12}
(2, 1, 2, 0)
{1, 2}
(2, 2, 2, 0)
{2}
(3, 3, 3, 0)
{3}
(3, 6, 6, 0)
{3, 6}
(3, 9, 9, 0)
{9, 3}
(4, 4, 4, 0)
{4}
(6, 6, 6, 0)
{6}
(6, 6, 15, 0)
{6, 15}
(6, 15, 6, 0)
{6, 15}
(6, 15, 15, 0)
{6, 15}
(7, 1, 1, 0)
{1, 7}
(7, 7, 7, 0)
{7}
(8, 1, 8, 0)
{8, 1}
(8, 8, 1, 0)
{8, 1}
(8, 8, 8, 0)
{8}
(9, 9, 3, 0)
{9, 3}
(9, 9, 9, 0)
{9}
(9, 9, 15, 0)
{9, 15}
(9, 15, 9, 0)
{9, 15}
(12, 1, 12, 0)
{1, 12}
(12, 12, 12, 0)
{12}
(14, 14, 14, 0)
{14}
(15, 6, 6, 0)
{6, 15}
(15, 9, 9, 0)
{9, 15}
(15, 9, 15, 0)
{9, 15}
(15, 15, 15, 0)
{15}
(5, 5, 5, 0)
{5}
(5, 12, 12, 0)
{12, 5}
(5, 12, 21, 0)
{21, 12, 5}
(5, 17, 5, 0)
{17, 5}
(5, 21, 21, 0)
{21, 5}
(6, 6, 6, 0)
{6}
(9, 9, 9, 0)
{9}
(12, 9, 9, 0)
{9, 12}
(12, 12, 12, 0)
{12}
(12, 13, 13, 0)
{12, 13}
(13, 6, 13, 0)
{13, 6}
(13, 13, 13, 0)
{13}
(13, 21, 13, 0)
{21, 13}
(13, 21, 24, 0)
{24, 21, 13}
(15, 15, 15, 0)
{15}


### Fine context condition

In [285]:
import torch
import pandas as pd
import random

# Assuming these variables are already defined: datasets, paths, setting, non_default_gran_list, n_epochs, n_values, n_attributes


# Go through all datasets
for i, d in enumerate(datasets):
    path_to_run = paths[i] + '/' + str(setting) + '/granularity_fine' + '/' + str(0) + '/'
    
    path_to_interaction_train = path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0'
    path_to_interaction_val = path_to_run + 'interactions/validation/epoch_' + str(n_epochs) + '/interaction_gpu0'
    interaction = torch.load(path_to_interaction_train)
    #print("Interaction train path:", path_to_interaction_train)

    messages = interaction.message.argmax(dim=-1)
    messages = [msg.tolist() for msg in messages]
    sender_input = interaction.sender_input
    #print("Sender input shape:", sender_input.shape)
    n_targets = int(sender_input.shape[1] / 2)
    
    # Get target objects and fixed vectors to re-construct concepts
    target_objects = sender_input[:, :n_targets]
    target_objects = k_hot_to_attributes(target_objects, n_values[i])
    # Concepts are defined by a list of target objects (here one sampled target object) and a fixed vector
    (objects, fixed) = retrieve_concepts_sampling(target_objects, all_targets=True)
    concepts = list(zip(objects, fixed))
    #print(concepts[0])

   # Collect data for the DataFrame

    mess = []
    concept_reps= []

    for idx, (t_objects, t_fixed) in enumerate(concepts):
        #concept_sample = (tuple(t_objects[0]), tuple(t_fixed))
        concept_sample = (
                list(int(x) for x in t_objects[0]), 
                list(int(x) for x in t_fixed)
            )
        mess.append(tuple(messages[idx]))
        concept_reps.append(create_alternative_representation(concept_sample))

        
        

    # Create the DataFrame
    df = pd.DataFrame({'concepts': concept_reps, 'message': mess})


    # Group by messages and aggregate concepts
    grouped_df = df.groupby('message', as_index=False).agg({'concepts': lambda x: list(x)})
    synonym_df = df.groupby('concepts', as_index=False).agg({'message': lambda x:list(x)})
    synonym_df['message'] = synonym_df['message'].apply(lambda x: str(list(set(x))))

    

    # Display the 
    print(grouped_df.shape)
    print(synonym_df.shape)
    print('\n')
    grouped_df.to_csv('analysis/messages/'+str(d) + '_' + str(setting) + '_granularity_fine.csv', index=False)
    synonym_df.to_csv('analysis/messages/'+'synonyms_'+str(d) + '_' + str(setting) + '_granularity_fine.csv', index=False)


    ## create clean dataframe, where each concepts appears only once per row and always with same sample for easier comparison


(133, 2)
(99, 2)


(438, 2)
(582, 2)


(1308, 2)
(3929, 2)


(1071, 2)
(499, 2)


(6636, 2)
(5248, 2)


(4337, 2)
(2499, 2)




### D(3,4)

In [131]:
import ast
import pandas as pd

# Load DataFrame from CSV file
df = pd.read_csv('analysis/messages/(3,4)_standard_granularity_fine.csv')


## it looks loke this message referes to fixed 0 in first position whaaaaat

((1, 1, 0), (1, 1, 1))
((2, 1, 0), (1, 1, 1))


In [221]:
import ast
import pandas as pd

# Load DataFrame from CSV file
df = pd.read_csv('analysis/messages/(3,4)_standard_granularity_fine.csv')

for c in concepts_for_message(df, [(13,13,13,0),]):
    print(c)


(('?', 1, 2), (0, 1, 1))


In [235]:
df = pd.read_csv('analysis/messages/(3,4)_standard_granularity_fine.csv')
concepts = []
mess = []
lens = []
for row in df.itertuples():
    concepts.append(concepts_for_message(df, [row[1]]))
    lens.append(len(concepts_for_message(df, [row[1]])))
    mess.append(row[1])
    print(row[1], 'n_concepts', len(concepts_for_message(df, [row[1]])),)

res = pd.DataFrame({'message':mess, 'n_concepts': lens })
res.to_csv('analysis/messages/lens.csv', index=False)

(1, 1, 1, 0) n_concepts 1
(1, 1, 2, 0) n_concepts 1
(1, 1, 4, 0) n_concepts 1
(1, 1, 11, 0) n_concepts 1
(1, 1, 13, 0) n_concepts 1
(1, 1, 14, 0) n_concepts 1
(1, 2, 13, 0) n_concepts 1
(1, 13, 2, 0) n_concepts 1
(1, 13, 13, 0) n_concepts 1
(1, 14, 14, 0) n_concepts 2
(1, 15, 1, 0) n_concepts 1
(1, 15, 14, 0) n_concepts 1
(2, 2, 1, 0) n_concepts 1
(2, 2, 2, 0) n_concepts 2
(2, 2, 10, 0) n_concepts 1
(2, 2, 13, 0) n_concepts 1
(2, 4, 4, 0) n_concepts 1
(2, 10, 4, 0) n_concepts 1
(3, 3, 8, 0) n_concepts 1
(3, 3, 11, 0) n_concepts 1
(3, 3, 12, 0) n_concepts 1
(3, 3, 13, 0) n_concepts 1
(3, 5, 3, 0) n_concepts 1
(3, 5, 5, 0) n_concepts 1
(3, 5, 12, 0) n_concepts 1
(3, 12, 12, 0) n_concepts 1
(3, 13, 4, 0) n_concepts 1
(3, 13, 5, 0) n_concepts 1
(4, 3, 4, 0) n_concepts 1
(4, 3, 5, 0) n_concepts 1
(4, 3, 8, 0) n_concepts 1
(4, 4, 4, 0) n_concepts 1
(4, 5, 14, 0) n_concepts 1
(4, 8, 5, 0) n_concepts 1
(4, 8, 8, 0) n_concepts 1
(4, 10, 4, 0) n_concepts 1
(4, 13, 4, 0) n_concepts 1
(4, 13, 13, 

### D(3,8)

In [241]:
df = pd.read_csv('analysis/messages/(3,8)_standard_granularity_fine.csv')
concepts = []
mess = []
lens = []
for row in df.itertuples():
    concepts.append(concepts_for_message(df, [row[1]]))
    lens.append(len(concepts_for_message(df, [row[1]])))
    mess.append(row[1])
    print(row[1], 'n_concepts', len(concepts_for_message(df, [row[1]])),)

res = pd.DataFrame({'message':mess, 'n_concepts': lens })
res.to_csv('analysis/messages/lens.csv', index=False)

(1, 1, 1, 0) n_concepts 4
(1, 2, 1, 0) n_concepts 1
(1, 2, 10, 0) n_concepts 2
(1, 2, 14, 0) n_concepts 2
(1, 2, 22, 0) n_concepts 1
(1, 2, 26, 0) n_concepts 1
(1, 2, 27, 0) n_concepts 2
(1, 10, 1, 0) n_concepts 2
(1, 10, 10, 0) n_concepts 3
(1, 10, 11, 0) n_concepts 4
(1, 10, 14, 0) n_concepts 2
(1, 10, 26, 0) n_concepts 1
(1, 11, 11, 0) n_concepts 5
(1, 13, 26, 0) n_concepts 1
(1, 14, 1, 0) n_concepts 2
(1, 14, 14, 0) n_concepts 2
(1, 20, 10, 0) n_concepts 1
(1, 20, 14, 0) n_concepts 3
(1, 20, 20, 0) n_concepts 1
(1, 20, 22, 0) n_concepts 3
(1, 21, 26, 0) n_concepts 1
(1, 22, 14, 0) n_concepts 3
(1, 22, 22, 0) n_concepts 3
(1, 23, 14, 0) n_concepts 2
(1, 26, 26, 0) n_concepts 2
(1, 27, 1, 0) n_concepts 5
(2, 1, 10, 0) n_concepts 1
(2, 2, 1, 0) n_concepts 2
(2, 2, 2, 0) n_concepts 3
(2, 2, 3, 0) n_concepts 1
(2, 2, 10, 0) n_concepts 4
(2, 2, 11, 0) n_concepts 1
(2, 2, 14, 0) n_concepts 1
(2, 2, 17, 0) n_concepts 2
(2, 2, 26, 0) n_concepts 2
(2, 2, 27, 0) n_concepts 3
(2, 3, 3, 0) n_co

### D(3,16)

In [258]:
df = pd.read_csv('analysis/messages/(3,16)_standard_granularity_fine.csv')
concepts = []
mess = []
lens = []
for row in df.itertuples():
    concepts.append(concepts_for_message(df, [row[1]]))
    lens.append(len(concepts_for_message(df, [row[1]])))
    mess.append(row[1])
    print(row[1], 'n_concepts', len(concepts_for_message(df, [row[1]])),)

res = pd.DataFrame({'message':mess, 'n_concepts': lens })
res.to_csv('analysis/messages/lens.csv', index=False)

(1, 1, 1, 0) n_concepts 4
(1, 1, 5, 0) n_concepts 3
(1, 1, 6, 0) n_concepts 1
(1, 1, 19, 0) n_concepts 1
(1, 1, 24, 0) n_concepts 1
(1, 1, 32, 0) n_concepts 1
(1, 1, 47, 0) n_concepts 1
(1, 2, 2, 0) n_concepts 10
(1, 2, 5, 0) n_concepts 7
(1, 2, 16, 0) n_concepts 6
(1, 2, 19, 0) n_concepts 11
(1, 3, 13, 0) n_concepts 2
(1, 3, 16, 0) n_concepts 4
(1, 5, 5, 0) n_concepts 1
(1, 6, 1, 0) n_concepts 8
(1, 6, 5, 0) n_concepts 1
(1, 6, 11, 0) n_concepts 1
(1, 6, 31, 0) n_concepts 1
(1, 12, 1, 0) n_concepts 3
(1, 12, 2, 0) n_concepts 2
(1, 12, 5, 0) n_concepts 5
(1, 12, 6, 0) n_concepts 7
(1, 12, 9, 0) n_concepts 11
(1, 12, 12, 0) n_concepts 1
(1, 13, 5, 0) n_concepts 2
(1, 13, 16, 0) n_concepts 3
(1, 16, 5, 0) n_concepts 4
(1, 16, 16, 0) n_concepts 2
(1, 16, 46, 0) n_concepts 1
(1, 19, 2, 0) n_concepts 2
(1, 19, 19, 0) n_concepts 4
(1, 24, 1, 0) n_concepts 1
(1, 29, 16, 0) n_concepts 1
(1, 40, 10, 0) n_concepts 1
(1, 46, 16, 0) n_concepts 4
(1, 47, 1, 0) n_concepts 1
(1, 47, 5, 0) n_concepts 

In [257]:
df = pd.read_csv('analysis/messages/(3,16)_standard_granularity_fine.csv')

for c in concepts_for_message(df, [(1,12,9,0),]):
    print(c)


((12, 4, 5), (1, 1, 1))
((10, 9, 6), (1, 1, 1))
((12, 4, 6), (1, 1, 1))
((10, 4, 6), (1, 1, 1))
((10, 4, 5), (1, 1, 1))
((12, 9, 5), (1, 1, 1))
((11, 4, 5), (1, 1, 1))
(('?', 4, 5), (0, 1, 1))
((11, 9, 5), (1, 1, 1))
(('?', 9, 6), (0, 1, 1))
((10, 9, 5), (1, 1, 1))


In [260]:
df = pd.read_csv('analysis/messages/(3,16)_standard_granularity_fine.csv')

for c in concepts_for_message(df, [(5,22,5,0),]):
    print(c)


((0, 2, 3), (1, 1, 1))
((1, 14, 13), (1, 1, 1))
((1, 14, 7), (1, 1, 1))
((1, 14, 14), (1, 1, 1))
((0, 14, 7), (1, 1, 1))
((7, 0, 3), (1, 1, 1))
((0, 14, 14), (1, 1, 1))
((1, 14, 3), (1, 1, 1))
((7, 14, 13), (1, 1, 1))
(('?', 14, 13), (0, 1, 1))
(('?', 2, 7), (0, 1, 1))
((7, 14, 3), (1, 1, 1))
(('?', 2, 1), (0, 1, 1))
((0, 14, '?'), (1, 1, 0))
(('?', 2, 3), (0, 1, 1))
(('?', 2, '?'), (0, 1, 0))
((9, 2, 7), (1, 1, 1))
((0, 15, 7), (1, 1, 1))
(('?', 14, 14), (0, 1, 1))
((9, 2, 1), (1, 1, 1))
((7, 14, 14), (1, 1, 1))
((0, 2, 7), (1, 1, 1))
((0, 2, 14), (1, 1, 1))
